In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import zipfile

In [3]:
%%time
def load_data(filename: str) -> pd.DataFrame:
    headers = ["p1", "p36", "p37", "p2a", "weekday(p2a)", "p2b", "p6", "p7", "p8", "p9", "p10", "p11", "p12",
               "p13a", "p13b", "p13c", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23",
               "p24", "p27", "p28", "p34", "p35", "p39", "p44", "p45a", "p47", "p48a", "p49", "p50a", "p50b",
               "p51", "p52", "p53", "p55a", "p57", "p58", "a", "b", "d", "e", "f", "g", "h", "i", "j", "k", "l",
               "n", "o", "p", "q", "r", "s", "t", "p5a"]
    empty_file_names = ["08.csv", "09.csv", "10.csv", "11.csv", "12.csv", "13.csv"]

    regions = {
        "PHA": "00",
        "STC": "01",
        "JHC": "02",
        "PLK": "03",
        "ULK": "04",
        "HKK": "05",
        "JHM": "06",
        "MSK": "07",
        "OLK": "14",
        "ZLK": "15",
        "VYS": "16",
        "PAK": "17",
        "LBK": "18",
        "KVK": "19",
    }
    # invert dictionary and add file extension
    inv_regions = {v + '.csv': k for k, v in regions.items()}

    all_data = pd.DataFrame()
    with zipfile.ZipFile(filename) as outer_zip:
        for outer_file_name in outer_zip.namelist():
            with zipfile.ZipFile(outer_zip.open(outer_file_name)) as inner_zip:
                for inner_file_name in inner_zip.namelist():
                    with inner_zip.open(inner_file_name) as inner_file:
                        if inner_file_name not in empty_file_names:
                            if inner_file_name != 'CHODCI.csv':
                                data = pd.read_csv(inner_file, encoding='cp1250', sep=';', names=headers,
                                                   low_memory=False)
                                # find region by value in dict
                                data['region'] = inv_regions[inner_file_name]
                                all_data = pd.concat([all_data, data])
    return all_data

data_frame = load_data("data/data.zip")

CPU times: user 5.32 s, sys: 2.14 s, total: 7.46 s
Wall time: 7.6 s


In [4]:
%%time
def parse_data(df: pd.DataFrame, verbose: bool = False) -> pd.DataFrame:
    if verbose:
        print(f'orig_size={df.memory_usage(deep=True).sum() / 10 ** 6:.2f} MB')

    new_df = df.copy()

    # new_df['p1'] = new_df['p1'].astype('i', errors='ignore')
    integer_cols = ['p2b', 'p13a', 'p13b', 'p13c', 'p14', 'p45a', 'p47', 'p53']

    for col in integer_cols:
        new_df[col] = new_df[col].astype(int, errors='ignore')
    #
    # float_bad_cols = ['d', 'e']
    # for col in float_bad_cols:
    #

    float_cols = ['d', 'e']
    for col in float_cols:
        new_df[col] = new_df[col].str.replace(',', '.')
    #     new_df[col] = new_df[col].astype('f', errors='ignore')
    new_df['d'] = new_df.d.replace('', np.nan).astype(float, errors='ignore')
    new_df['e'] = new_df.e.replace('', np.nan).astype(float, errors='ignore')
    new_df['d'] = pd.to_numeric(new_df['d'], errors='coerce')
    new_df['e'] = pd.to_numeric(new_df['e'], errors='coerce')

    new_df['p13a'] = pd.to_numeric(new_df['p13a'], errors='coerce')
    new_df['p13b'] = pd.to_numeric(new_df['p13b'], errors='coerce')
    new_df['p13c'] = pd.to_numeric(new_df['p13c'], errors='coerce')

    # new_df = new_df.astype({'d': 'float', 'e': 'float'}, errors='ignore')

    category_cols = ['p12', 'p36', 'p37', 'weekday(p2a)', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11',
                     'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p27', 'p28', 'p34', 'p35',
                     'p44', 'p45a', 'p48a', 'p49', 'p50a', 'p50b', 'p51', 'p52', 'p55a', 'p57', 'p58', 'a', 'b', 'h',
                     'i', 'j', 'k', 'l', 'o', 'p', 'q', 'r', 's', 't', 'p5a', 'f', 'g', 'n']
    for col in category_cols:
        new_df[col] = new_df[col].astype('category')

    new_df['date'] = pd.to_datetime(new_df['p2a']).astype('datetime64[ns]')
    new_df = new_df.drop_duplicates('p1')

    if verbose:
        print(f'new_size={new_df.memory_usage(deep=True).sum() / 10 ** 6:.2f} MB')

    new_df.info(verbose=True)

    return new_df

df = parse_data(data_frame)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609146 entries, 0 to 2465
Data columns (total 66 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   p1            609146 non-null  int64         
 1   p36           609146 non-null  category      
 2   p37           282123 non-null  category      
 3   p2a           609146 non-null  object        
 4   weekday(p2a)  609146 non-null  category      
 5   p2b           609146 non-null  int64         
 6   p6            609146 non-null  category      
 7   p7            609146 non-null  category      
 8   p8            609146 non-null  category      
 9   p9            609146 non-null  category      
 10  p10           609146 non-null  category      
 11  p11           609146 non-null  category      
 12  p12           609146 non-null  category      
 13  p13a          609146 non-null  int64         
 14  p13b          609146 non-null  int64         
 15  p13c          60914

In [ ]:
def plot_consequences(df: pd.DataFrame, fig_location: str = None, show_figure: bool = False):
    new_df = df.copy()
    selected_regions = ['HKK', 'ZLK', 'JHC', 'PLK']
    # selected_regions = ['PHA', 'STC', 'JHC', 'PLK']
    new_df = new_df[new_df['region'].isin(selected_regions)]

    new_df.loc[new_df['p13c'] > 0, 'lehke'] = 1
    new_df.loc[new_df['p13b'] > 0, 'tezke'] = 1
    new_df.loc[new_df['p13a'] > 0, 'smrt'] = 1

    new_df.loc[new_df['tezke'] > 0, 'lehke'] = 0
    new_df.loc[new_df['smrt'] > 0, 'tezke'] = 0

    new_df.loc[new_df['smrt'] > 0, 'Nasledky'] = 'usmrceni'
    new_df.loc[new_df['tezke'] > 0, 'Nasledky'] = 'tezke zraneni'
    new_df.loc[new_df['lehke'] > 0, 'Nasledky'] = 'lehke zraneni'

    new_df = new_df[new_df['Nasledky'].notna()]

    new_df = new_df[['date', 'region', 'Nasledky']]
    new_df = new_df.groupby(['date', 'region'])['Nasledky'].count() #.reset_index()
    new_df = new_df.groupby('region').resample("MS", on='date').sum().reset_index()


    # new_df = new_df.melt(id_vars=['region', 'date'], var_name='Nasledky', value_name='pocet')

    # new_df = new_df.groupby(['region', 'month']).agg({'p13a': 'sum', 'p13b': 'sum', 'p13c': 'sum'}).reset_index()
    # new_df = pd.pivot_table(new_df, index=['p13a', 'p13b', 'p13c'])
    # new_df = pd.pivot_table(new_df, values=['p13a', 'p13b', 'p13c'],
    #                         index=['region', 'date'],
    #                         # columns=
    #                         aggfunc=np.sum
    #                         )
    # print(new_df)
    # # for r in selected_regions:
    # new_df = new_df.unstack('region').resample('MS').sum().stack('region')
    #
    # new_df['p13a'] = new_df['p13a'].astype(int, errors='ignore')
    # new_df['p13b'] = new_df['p13b'].astype(int, errors='ignore')
    # new_df['p13c'] = new_df['p13c'].astype(int, errors='ignore')
    # # new_df = new_df.stack()
    #
    # print(new_df)
    # new_df.info()
    # # new_df = new_df.stack(0, )
    #
    # g = sns.lineplot(data=new_df.stack(),
    #                  # x='date',
    #                  # y='p13a',
    #                  # col='region',
    #                  # col_order=selected_regions,
    #                  # hue='region',
    #                  # hue_order=['bocni', 'zezadu', 'celni'],
    #                  # col_wrap=2,
    #                  k
    #                  palette='muted',
    #                  # sharex=False,
    #                  )

    if fig_location is not None:
        plt.savefig(fig_location)

    if show_figure:
        plt.show()
    pass
